In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd

driver = webdriver.Chrome(ChromeDriverManager().install())

tabela_produtos = pd.read_excel("buscas.xlsx")
display(tabela_produtos)

In [ ]:
produto = 'iphone 12 64 gb'
termos_banidos = '11 13 14 mini watch'

preco_minimo = 3000
preco_maximo = 3500

In [ ]:
def busca_google_shopping(driver, produto, termos_banidos, preco_minimo, preco_maximo):

    produto.lower()
    termos_banidos.lower()

    lista_nomes_produto = produto.split(" ")
    lista_termos_banidos = termos_banidos.split(" ")

    lista_itens_encontrados = []

    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)

    driver.get("https://www.google.com.br/")

    driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(produto, Keys.ENTER)

    elementos = driver.find_elements(By.CLASS_NAME, 'hdtb-mitem')

    for elemento in elementos:
        if "Shopping" in elemento.text:
            elemento.click()
            break

    lista_resultados = driver.find_elements(By.CLASS_NAME, 'i0X6df')

    for resultado in lista_resultados:
        nome = resultado.find_element(By.CLASS_NAME, 'tAxDx').text.lower()

        tem_termos_banidos = False
        for termo in lista_termos_banidos:
            if termo in nome:
                tem_termos_banidos = True

        tem_lista_nomes_produto = True
        for termo in lista_nomes_produto:
            if termo not in nome:
                tem_lista_nomes_produto = False

        if tem_termos_banidos == False and tem_lista_nomes_produto:
            preco = resultado.find_element(By.CLASS_NAME, 'a8Pemb').text.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
            preco = float(preco)

            if preco_minimo <= preco <= preco_maximo:

                child = resultado.find_element(By.CLASS_NAME, 'XrAfOe')
                parent = child.find_element(By.XPATH, '..')
                link = parent.get_attribute('href')

                lista_itens_encontrados.append((nome, preco, link))
    return lista_itens_encontrados



    def busca_buscape(driver, produto, termos_banidos, preco_minimo, preco_maximo):
        produto.lower()
        termos_banidos.lower()

        lista_nomes_produto = produto.split(" ")
        lista_termos_banidos = termos_banidos.split(" ")

        lista_itens_encontrados = []   
    
        preco_minimo = float(preco_minimo)
        preco_maximo = float(preco_maximo)

        driver.get('https://www.buscape.com.br/')

        driver.find_element(By.XPATH, '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(produto, Keys.ENTER)

In [ ]:
lista_ofertas_google_shopping = busca_google_shopping(driver, produto, termos_banidos, preco_minimo, preco_maximo)
print(lista_ofertas_google_shopping)